<a href="https://colab.research.google.com/github/jeromepatel/DL-University-Course-Practicals/blob/master/18BCE088_DL_practical_5_TL_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Feb 24 15:49:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

#importing libraries
from keras.datasets.cifar10 import load_data
# from matplotlib import pyplot
from tensorflow import keras
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D, BatchNormalization ,GlobalAveragePooling2D
from keras import backend as K
#from keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.xception import Xception, preprocess_input
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

import numpy as np
# batch_size = 512
# num_classes = 10
# epochs = 30


In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
(X_train_data, Y_train), (X_test_data, Y_test) = load_data()
X_train_data.shape, X_test_data.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [ ]:
# X_train = X_train_data/255
# X_test = X_test_data/255

# del X_train_data, X_test_data

In [ ]:
%%time
import cv2
rows = 71
columns  = 71
X_train_1 = np.array([cv2.resize(i, dsize=(rows, columns), interpolation=cv2.INTER_CUBIC) for i in X_train_data])
X_train = preprocess_input(X_train_1)
X_test_1 = np.array([cv2.resize(i, dsize=(rows, columns), interpolation=cv2.INTER_CUBIC) for i in X_test_data])
X_test = preprocess_input(X_test_1)

del X_train_data, X_test_data, X_train_1,X_test_1

num_classes = 10
img_rows, img_cols = 32, 32
input_shape = (img_rows, img_cols, 3)
y_train,y_test = Y_train,Y_test
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes)
Y_test = tf.keras.utils.to_categorical(Y_test, num_classes)
del y_test,y_train

CPU times: user 3.98 s, sys: 1.36 s, total: 5.34 s
Wall time: 5.34 s


In [ ]:
X_train.shape

(50000, 71, 71, 3)

In [ ]:
base_model = Xception(include_top=False, weights='imagenet', input_shape=(rows, columns, 3), pooling=None)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
base_model.summary()

In [ ]:
%%time
from tensorflow.keras import layers
from tensorflow.keras import Model

#base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(100,100,3),pooling = None)

# x_train,x_test = X_train,X_test
# X_train = base_model.predict(X_train)
# del x_train
# X_test = base_model.predict(X_test)
# del x_test


flat1 = Flatten()(base_model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
class2 = Dropout(0.4)(class1)
output = Dense(10, activation='softmax')(class2)
# define new model
model = Model(inputs=base_model.inputs, outputs=output)

model.compile(optimizer = Adam(lr = 0.001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])


training_history = None
batch_size = 128
epochs = 20
try:
   training_history =  model.fit(
    X_train,
        Y_train,
        validation_data = (X_test,Y_test),
        batch_size = batch_size,
        epochs = epochs,
        shuffle = True,
        verbose = 1
    )
except KeyboardInterrupt as e:
    print("\njust wait to complete training!!\n")

Epoch 1/20
391/391 [==============================] - 38s 90ms/step - loss: 2.0700 - acc: 0.6285 - val_loss: 0.7789 - val_acc: 0.7335
Epoch 2/20
391/391 [==============================] - 33s 85ms/step - loss: 0.7849 - acc: 0.7319 - val_loss: 0.7340 - val_acc: 0.7469
Epoch 3/20
391/391 [==============================] - 33s 85ms/step - loss: 0.7127 - acc: 0.7555 - val_loss: 0.7234 - val_acc: 0.7566
Epoch 4/20
391/391 [==============================] - 34s 87ms/step - loss: 0.6687 - acc: 0.7674 - val_loss: 0.7041 - val_acc: 0.7613
Epoch 5/20
391/391 [==============================] - 33s 85ms/step - loss: 0.6173 - acc: 0.7841 - val_loss: 0.7035 - val_acc: 0.7616
Epoch 6/20
391/391 [==============================] - 33s 86ms/step - loss: 0.5846 - acc: 0.7966 - val_loss: 0.7356 - val_acc: 0.7601
Epoch 7/20
391/391 [==============================] - 34s 86ms/step - loss: 0.5609 - acc: 0.8017 - val_loss: 0.7137 - val_acc: 0.7651
Epoch 8/20
391/391 [==============================] - 33s 86ms

In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((50000, 71, 71, 3), (10000, 71, 71, 3), (50000, 10), (10000, 10))

In [ ]:
# model = Sequential()
# model.add(GlobalAveragePooling2D(input_shape = (1,1,2048)))
# model.add(Dropout(0.4))
# #model.add(Dense(256,input_dim = X_train.shape[1],activation='relu'))
# model.add(Dense(512,activation = 'relu'))
# model.add(Dropout(0.5))
# #model.add(BatchNormalization())
# # model.add(Dense(128,activation = 'relu'))
# # model.add(Dropout(0.5))
# model.add(Dense(Y_test.shape[1], activation='softmax'))

In [ ]:
# model = models.Sequential()
# model.add(GlobalAveragePooling2D(input_shape=(7, 7, 2048)))
# model.add(Dropout(0.2))
# model.add(Dense(Y_test.shape[1], activation='softmax'))

In [ ]:
# model.compile(Adam(lr = 0.001),
#               loss = 'categorical_crossentropy',
#               metrics = ['accuracy'])

In [ ]:
# %%time
# training_history = None
# batch_size = 128
# epochs = 20
# try:
#    training_history =  model.fit(
#     X_train,
#         Y_train,
#         validation_data = (X_test,Y_test),
#         batch_size = batch_size,
#         epochs = epochs,
#         shuffle = True,
#         verbose = 1
#     )
# except KeyboardInterrupt as e:
#     print("\njust wait to complete training!!\n")

Epoch 1/20
391/391 [==============================] - 2s 4ms/step - loss: 1.7619 - accuracy: 0.4499 - val_loss: 0.9857 - val_accuracy: 0.6788
Epoch 2/20
391/391 [==============================] - 1s 4ms/step - loss: 1.0794 - accuracy: 0.6432 - val_loss: 0.9428 - val_accuracy: 0.6861
Epoch 3/20
391/391 [==============================] - 1s 4ms/step - loss: 1.0294 - accuracy: 0.6579 - val_loss: 0.9201 - val_accuracy: 0.6957
Epoch 4/20
391/391 [==============================] - 1s 4ms/step - loss: 1.0175 - accuracy: 0.6616 - val_loss: 0.9122 - val_accuracy: 0.6996
Epoch 5/20
391/391 [==============================] - 1s 4ms/step - loss: 1.0009 - accuracy: 0.6702 - val_loss: 0.9305 - val_accuracy: 0.6934
Epoch 6/20
391/391 [==============================] - 1s 4ms/step - loss: 1.0091 - accuracy: 0.6675 - val_loss: 0.9207 - val_accuracy: 0.6988
Epoch 7/20
391/391 [==============================] - 1s 4ms/step - loss: 1.0031 - accuracy: 0.6679 - val_loss: 0.9229 - val_accuracy: 0.6920
Epoch 

In [ ]:
#@title Default title text
# X_train = X_train/255   #in range (0,1) now 
# X_test = X_test/255
# print(X_train[0].shape)
# Y_train[0]

In [ ]:
#@title Default title text
# img_rows, img_cols = 32, 32
# y_train,y_test = Y_train,Y_test
# input_shape = (img_rows, img_cols, 3)
# # convert class vectors to binary class matrices
# Y_train = tf.keras.utils.to_categorical(Y_train, num_classes)
# Y_test = tf.keras.utils.to_categorical(Y_test, num_classes)

In [ ]:
#prediction using test set
prediction = model.predict(X_test)
y_test = np.argmax(Y_test, axis=1)
prediction = np.argmax(prediction, axis=1)
print('\nAccuracy:', accuracy_score(y_test, prediction, normalize=True))
print('\nClassification Report:' + classification_report(y_test, prediction) + '\nConfusion Matrix:')
print(confusion_matrix(y_test, prediction))


Accuracy: 0.7692

Classification Report:              precision    recall  f1-score   support

           0       0.79      0.80      0.79      1000
           1       0.89      0.81      0.85      1000
           2       0.71      0.73      0.72      1000
           3       0.64      0.58      0.61      1000
           4       0.69      0.74      0.72      1000
           5       0.71      0.72      0.72      1000
           6       0.77      0.76      0.77      1000
           7       0.82      0.79      0.81      1000
           8       0.85      0.86      0.86      1000
           9       0.81      0.89      0.84      1000

    accuracy                           0.77     10000
   macro avg       0.77      0.77      0.77     10000
weighted avg       0.77      0.77      0.77     10000

Confusion Matrix:
[[802  11  32  15  14   2   6  13  75  30]
 [ 24 815   0   7   1  10   5   3  17 118]
 [ 43   1 730  38  84  33  48  18   4   1]
 [ 18   7  62 584  54 156  64  32  12  11]
 [ 23   1 

In [ ]:
# import cv2
# from numpy import array, argmax
# from sklearn import metrics
# import matplotlib.pyplot as plt
# from tensorflow.keras import datasets, utils, models, optimizers
# from tensorflow.keras.applications.xception import Xception, preprocess_input
# from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
# (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# # print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# x_train = array([cv2.resize(i, dsize=(72, 72), interpolation=cv2.INTER_CUBIC) for i in x_train])
# x_train = preprocess_input(x_train)
# x_test = array([cv2.resize(i, dsize=(72, 72), interpolation=cv2.INTER_CUBIC) for i in x_test])
# x_test = preprocess_input(x_test)

# y_train = utils.to_categorical(y_train)
# y_test = utils.to_categorical(y_test)
# model_xception = Xception(include_top=False, weights='imagenet', input_shape=(72, 72, 3), pooling=None)

# del y_train,y_test
# #model_xception.summary()

# x_train = model_xception.predict(x_train)
# x_test = model_xception.predict(x_test)
# model = models.Sequential()
# model.add(GloalAveragePooling2D(input_size=(7, 7, 2048)))
# model.add(Dropout(0.3))
# model.add(Dense(x_test.shape[1], activaton='softmax'))

# # model.summary()
# model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=32, epochs=10, verbose = True)
# prediction = model.predict(x_test)

# print(prediction)

# y_test = argmax(y_test, axis=1)
# prediction = argmax(prediction, axis=1)

# print('\nAccuracy:', metrics.accuracy_score(y_test, prediction, normalize=True))
# print('\nClassification Report:' + metrics.classification_report(y_test, prediction) + '\nConfusion Matrix:')
# print(metrics.confusion_matrix(y_test, prediction))

In [ ]:
lst = []
i =1
while 1:
    i +=1
    lst.append('Hello'*i)